In [1]:
%matplotlib inline
import pandas as pd
import os
import numpy as np

## Cleaning

In [25]:
# TODO(run clean)
hash2 = lambda x: hash(x) % 10**4

def findFirstVal(s):
    for i in s:
        if i != None:
            return i

def transform_df(df, name):
    pass

def find_columns_type(df):
    result = {}
    for key, val in df.iteritems():
        for i in val:
            if i != None:
                result[key] = type(i)
                break 
    return result
       
def append_none(d):
    for k, v in d.iteritems():
        d[k].append(None)
        
def transform_dict(series, key_old):
    data = {}
    for item in series:
        if type(item) == dict:
            for k, v in item.iteritems():
                data[k] = []
    for item in series:
        if item == None:
            append_none(data)
        elif type(item) == dict:
            for k in data.keys():
                v = None if k not in item else item[k]
                if type(v) == dict:
                    data[k].append(hash2(None))
                elif type(v) == list:
                    data[k].append(hash2(None))
                elif type(v) in [int, float]:
                    data[k].append(v)
                else:
                    data[k].append(hash2(v))
    return {key_old + '++' + k : v for k, v in data.iteritems()}

def clean_val(df, columns_map):
    result = {}
    for key, val in df.iteritems():
        if not key in columns_map or columns_map[key] == list:
            continue
        elif columns_map[key] in [int, float]:
            result[key] = val
        elif columns_map[key] == dict:
            result.update(transform_dict(val, key))
        else:
#             else columns_map[key] in [str, list]:
            result[key] = map(hash2, val)
    return result

# Public api, use this!
def clean_dataset(data):
    columns_types = find_columns_type(data)
    clean_df = clean_val(data, columns_types)
    ret = pd.DataFrame(clean_df)
    ret.fillna(0, inplace=True)
    return ret

In [9]:
pdX = clean_dataset(pd.read_pickle(os.path.join('..', 'data', 'xtrain.pkl')))
y = pd.read_pickle(os.path.join('..', 'data', 'ytrain.pkl'))

# y has one more predicion - trim it
y = y[0:-1]

x72496 x32520 x46219 x97734 x98434 x13664 x83336 x48420 x97674 agent_type api_call_datetime browser browser_language browser_string browser_string_hash browser_string_mismatch browser_version cc_bin_number cc_bin_number_brand cc_bin_number_category cc_bin_number_geo cc_bin_number_org cc_bin_number_type cc_number_hash_assert_history cc_number_hash_first_seen cc_number_hash_last_event cc_number_hash_last_update cc_number_hash_result cc_number_hash_score cc_number_hash_worst_score cidr_number x21877 custom_count_1 custom_count_2 custom_count_3 custom_count_4 x87918 x68280 x31255 x85112 x18591 x35653 x75049 x76409 x80911 x96255 x60781 x23739 dns_ip_city dns_ip_geo dns_ip_isp dns_ip_latitude dns_ip_longitude dns_ip_organization dns_ip_region dns_ip_region_iso_code x5290 x5170 x5342 x5193 x92645 x31903 x94347 x82598 x40007 x13872 x59752 x14458 x85212 x54084 x3314 headers_name_value_hash headers_order_string_hash x10047 x80385 input_ip_assert_history input_ip_attributes input_ip_city input_ip

In [11]:
print(pdX.shape)
print(len(y)) # y is 1-dim

(17253, 337)
17253


In [40]:
from sklearn.cross_validation import train_test_split
from sklearn.metrics import roc_auc_score, f1_score
import xgboost as xg

# suggested params
param = {
    'bst:eta' : [0.002, 0.02],
    'bst:subsample' : [0.5,1],
    'bst:mim_child_weight' : [0.1,10],
    'bst:max_depth' : [2,30],
    'bst:gamma' : [0,1],
    'objective':'binary:logistic'
}

X_tmp, X_eval, y_tmp, y_eval = train_test_split(
    pdX, y, test_size=0.2, random_state=42
)

X_fit, X_test, y_fit, y_test = train_test_split(
    X_tmp, y_tmp, test_size=0.2, random_state=42
)

# TODO(anyone): set up parameter random search

tb = xg.XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0, learning_rate=0.15, max_delta_step=0, max_depth=20,
       min_child_weight=1, missing=None, n_estimators=150, nthread=-1,
       objective='binary:logistic', reg_alpha=0.1, reg_lambda=1.1,
       scale_pos_weight=1, seed=0, silent=True, subsample=0.8)
tb.fit(X_fit, y_fit, eval_set=[(X_eval, y_eval)], eval_metric='logloss', verbose=False)

# print tb.evals_result()
y_pred = tb.predict_proba(X_test)

In [41]:
print len(y_pred)
# print f1_score(y_eval, y_pred)
print len(filter(lambda x: x, map(lambda x: x[1] > 0.07, y_pred)))

2761
38


## Test set

In [42]:
# auc
from sklearn.metrics import roc_auc_score
y_true = y_test
y_scores = map(lambda x: x[1], y_pred)
roc_auc_score(y_true, y_scores)

0.74268904423928706

## Predictions for real testset

In [16]:
len(X_test.columns)

337

In [17]:
X_test_real = clean_dataset(pd.read_pickle(os.path.join('..', 'data', 'xtest.pkl')))
print(X_test_real.shape)


x72496 x32520 x46219 x97734 x98434 x13664 x83336 x48420 x97674 agent_type api_call_datetime browser browser_language browser_string browser_string_hash browser_string_mismatch browser_version cc_bin_number cc_bin_number_brand cc_bin_number_category cc_bin_number_geo cc_bin_number_org cc_bin_number_type cc_number_hash_assert_history cc_number_hash_first_seen cc_number_hash_last_event cc_number_hash_last_update cc_number_hash_result cc_number_hash_score cc_number_hash_worst_score cidr_number x21877 custom_count_1 custom_count_2 custom_count_3 custom_count_4 x87918 x68280 x31255 x85112 x18591 x35653 x75049 x76409 x80911 x96255 x60781 x23739 dns_ip_city dns_ip_geo dns_ip_isp dns_ip_latitude dns_ip_longitude dns_ip_organization dns_ip_region dns_ip_region_iso_code x5290 x5170 x5342 x5193 x92645 x31903 x94347 x82598 x40007 x13872 x59752 x14458 x85212 x54084 x3314 headers_name_value_hash headers_order_string_hash x10047 x80385 input_ip_assert_history input_ip_attributes input_ip_city input_ip

In [19]:
X_test_real_match = X_test_real[X_test.columns]
len(X_test_real_match.columns)

337

In [20]:
y_test_real = tb.predict_proba(X_test_real_match)

In [21]:
y_test_real

array([[ 0.95795226,  0.04204771],
       [ 0.98484695,  0.01515303],
       [ 0.96372944,  0.03627057],
       ..., 
       [ 0.99053693,  0.00946308],
       [ 0.9598124 ,  0.04018762],
       [ 0.98902661,  0.01097341]], dtype=float32)

In [22]:
# write results
import pickle
with open(os.path.join('..','data', 'ytest.pkl'), 'w') as f:
    pickle.dump(y_test_real,f)

In [26]:
a = pd.read_pickle(os.path.join('..','data', 'ytest.pkl'))
len(a)

7395